# Laboratorio 7
## Clasificación de Malware
Clasificación de malware con identificación de imágenes (TF & Keras)
> #### Gerardo Méndez 18239

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.constraints import maxnorm

### PARTE 1
Carga de Imágenes & Preprocesamiento

#### Carga de datasets

In [17]:
batch_size = 32
img_height = 128
img_width = 128

In [18]:
DATA_DIRECTORY = 'data/images'

In [19]:
dataset = tf.keras.utils.image_dataset_from_directory(
  DATA_DIRECTORY,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 9339 files belonging to 25 classes.


In [12]:
class_names = dataset.class_names
print(class_names)

['Adialer.C', 'Agent.FYI', 'Allaple.A', 'Allaple.L', 'Alueron.gen!J', 'Autorun.K', 'C2LOP.P', 'C2LOP.gen!g', 'Dialplatform.B', 'Dontovo.A', 'Fakerean', 'Instantaccess', 'Lolyda.AA1', 'Lolyda.AA2', 'Lolyda.AA3', 'Lolyda.AT', 'Malex.gen!J', 'Obfuscator.AD', 'Rbot!gen', 'Skintrim.N', 'Swizzor.gen!E', 'Swizzor.gen!I', 'VB.AT', 'Wintrim.BX', 'Yuner.A']


#### Class distribution

In [13]:
import collections

def get_class_distribution(dataset):
    class_distribution = {}
    for element in dataset.as_numpy_iterator():
        label = element[1][0]

        try:
            class_distribution[label] += 1
        except:
            class_distribution[label] = 0

    class_distribution = collections.OrderedDict(sorted(class_distribution.items())) 
    return class_distribution

In [14]:
class_distribution = get_class_distribution(dataset)

In [15]:
for i in range(len(class_names)):
    name = class_names[i]
    dist = class_distribution[i]
    print(name, " " * (15 - len(name)), dist)

Adialer.C        121
Agent.FYI        115
Allaple.A        2948
Allaple.L        1590
Alueron.gen!J    197
Autorun.K        105
C2LOP.P          145
C2LOP.gen!g      199
Dialplatform.B   176
Dontovo.A        161
Fakerean         380
Instantaccess    430
Lolyda.AA1       212
Lolyda.AA2       183
Lolyda.AA3       122
Lolyda.AT        158
Malex.gen!J      135
Obfuscator.AD    141
Rbot!gen         157
Skintrim.N       79
Swizzor.gen!E    127
Swizzor.gen!I    131
VB.AT            407
Wintrim.BX       96
Yuner.A          799


### PARTE 2
Plot de Imágenes

In [20]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

NameError: name 'train_dataset' is not defined

<Figure size 720x720 with 0 Axes>

### PARTE 3
Construcción de Red Neuronal (CNN)

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (5, 5), input_shape=(128, 128, 3), activation='relu', padding='same'))
#model.add(keras.layers.MaxPooling2D(2))
#model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(len(class_names), activation='softmax'))

### PARTE 4
Resumen del modelo

In [ ]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.summary()

### PARTE 5
División de los datos

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
  DATA_DIRECTORY,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

In [ ]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
  DATA_DIRECTORY,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

### PARTE 6
Entrenamiento de modelos

In [ ]:
model.fit(train_dataset, epochs=10)

Epoch 1/10
